In [131]:
# import ting
import numpy as np
import pandas as pd
import geopandas as gpd

In [132]:
# load in the new dataframes we will be working with
crime = pd.read_csv('~/Downloads/opportunity mapper/cl_crime.csv')
rent_inc = pd.read_csv('~/Downloads/opportunity mapper/cl_rent_income.csv')
gdf_zip = gpd.read_file('~/Downloads/opportunity mapper/raw datasets/BayArea ShapeFile/s7888q.shp')
bart = pd.read_csv('~/Downloads/opportunity mapper/raw datasets/BartLatLong.csv')
caltr = pd.read_csv('~/Downloads/opportunity mapper/raw datasets/CalTrain.csv')

In [65]:
# How many zip codes are in both shapefile and rent_inc?
# Clean rent_inc and shapefile for easier merge to see zip codes in common
rent_inc = rent_inc.rename(columns={'ZIP_CODE': "ZIP"})
rent_inc['ZIP'] = rent_inc['ZIP'].astype(str)
gdf_zip['ZIP'] = gdf_zip['ZIP'].astype(str)

gdf_zip.shape # (187, 6)
rent_inc.shape # (290, 21)
gdf_zip.merge(rent_inc, how='inner', on="ZIP").shape #(184, 26)

# The shapefile with Bay Area zip codes inner joined on the rent_inc zip codes results
# in us having 184 zip codes to work with.

(184, 26)

In [133]:
# make transit df
# First, set some method to tell which rows of concatted df are BART or CalTrain
# (Ignore BART column, that's from original CSV and not relevant)
bart['IS_BART'] = True
caltr['IS_BART'] = False
transit = pd.concat([bart, caltr])
gdf_stations = gpd.GeoDataFrame(
    transit,
    geometry=gpd.points_from_xy(transit.X, transit.Y),
    crs="EPSG:4326"  # Standard Lat/Lon projection
)
gdf_stations = gdf_stations.to_crs(gdf_zip.crs) # use same coordinate system as zip codes
transit_join = gpd.sjoin(gdf_zip, gdf_stations, how='inner', predicate='contains') # find which zip code polygon contains which station
transit_join['BART_COUNT'] = transit_join['IS_BART'].astype(int) # convert boolean values to ints
transit_join['CalTrain_COUNT'] = transit_join['IS_BART'].astype(int)
transit_cnt = transit_join.groupby('ZIP').agg({'BART_COUNT': 'sum', 'CalTrain_COUNT': 'sum'}).reset_index()
transit_cnt['TOTAL_TRANSIT'] = transit_cnt['BART_COUNT'] + transit_cnt['CalTrain_COUNT']
transit_cnt # all transit zip codes have their totals available now!

,ZIP,BART_COUNT,CalTrain_COUNT,TOTAL_TRANSIT
0,94002,0,0,0
1,94010,0,0,0
2,94014,2,2,4
3,94025,0,0,0
4,94027,0,0,0
5,94030,1,1,2
6,94040,0,0,0
7,94063,0,0,0
8,94066,1,1,2
9,94070,0,0,0
